In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
# Load the iris dataset
df = pd.read_csv("../data/cleaned_data.csv", low_memory = False)



In [5]:


# Splitting the DataFrame into X (features) and y (labels)
X = df.drop(columns=['default'])  # Drop the 'label' column to get the features
y = df['default']  # Get only the 'label' column as the target variable

# Printing the first few rows of X and y for verification
print("Features (X):")
print(X.head())
print("\nLabels (y):")
print(y.head())


Features (X):
   Unnamed: 0       id  member_id  loan_amnt  funded_amnt  term  int_rate  \
0           0  1077501    1296599     5000.0       5000.0    36     10.65   
1           1  1077430    1314167     2500.0       2500.0    60     15.27   
2           2  1077175    1313524     2400.0       2400.0    36     15.96   
3           3  1076863    1277178    10000.0      10000.0    36     13.49   
4           4  1075358    1311748     3000.0       3000.0    60     12.69   

   installment  grade  sub_grade  ...  addr_state    dti  delinq_2yrs  \
0       162.87      2          7  ...           3  27.65          0.0   
1        59.83      3         14  ...          10   1.00          0.0   
2        84.33      3         15  ...          14   8.72          0.0   
3       339.31      3         11  ...           4  20.00          0.0   
4        67.79      2         10  ...          37  17.94          0.0   

   mths_since_last_delinq  total_acc  out_prncp   total_pymnt  \
0                  

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Define the XGBoost classifier
clf = xgb.XGBClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

In [8]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.989992896365871
